In [61]:
import importlib
import Alpha
importlib.reload(Alpha)
from Alpha import merge_df, completed

In [62]:
train = pd.read_csv("C:\\Users\\solba\\dacon-project\\data\\raw\\train.csv")
test = pd.read_csv("C:\\Users\\solba\\dacon-project\\data\\raw\\test.csv")

In [ ]:
train = merge_df(train)

In [64]:
data

,ID,school1,job,nationality,High Tech,Data Friendly,Others,hope_for_group,desired_career_path,incumbents_level,incumbents_lecture,incumbents_company_level,incumbents_lecture_type,incumbents_lecture_scale,count,time_input,want_count
0,TRAIN_000,22,0,1,0,1,1,1,0,0,0,16,1,2,3,2,8
1,TRAIN_001,1,0,1,0,0,1,0,3,1,7,6,0,5,1,3,10
2,TRAIN_002,27,0,1,0,0,1,0,0,1,5,4,0,5,2,10,12
3,TRAIN_003,1,0,1,0,0,1,1,3,1,7,6,1,5,3,2,7
4,TRAIN_004,16,0,1,1,0,1,0,3,1,5,4,1,5,2,2,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
743,TRAIN_743,13,0,1,1,1,0,0,3,1,5,4,1,5,3,3,6
744,TRAIN_744,43,0,1,1,0,0,1,3,1,5,4,1,2,3,3,6
745,TRAIN_745,11,0,1,0,1,1,0,0,0,7,16,1,2,1,1,5
746,TRAIN_746,30,0,1,1,1,0,2,3,1,7,4,1,5,1,2,8


In [65]:
Y = completed(train, data)

In [66]:
Y

,ID,school1,job,nationality,High Tech,Data Friendly,Others,hope_for_group,desired_career_path,incumbents_level,incumbents_lecture,incumbents_company_level,incumbents_lecture_type,incumbents_lecture_scale,count,time_input,want_count,completed
0,TRAIN_000,22,0,1,0,1,1,1,0,0,0,16,1,2,3,2,8,0
1,TRAIN_001,1,0,1,0,0,1,0,3,1,7,6,0,5,1,3,10,0
2,TRAIN_002,27,0,1,0,0,1,0,0,1,5,4,0,5,2,10,12,0
3,TRAIN_003,1,0,1,0,0,1,1,3,1,7,6,1,5,3,2,7,1
4,TRAIN_004,16,0,1,1,0,1,0,3,1,5,4,1,5,2,2,11,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
743,TRAIN_743,13,0,1,1,1,0,0,3,1,5,4,1,5,3,3,6,0
744,TRAIN_744,43,0,1,1,0,0,1,3,1,5,4,1,2,3,3,6,1
745,TRAIN_745,11,0,1,0,1,1,0,0,0,7,16,1,2,1,1,5,1
746,TRAIN_746,30,0,1,1,1,0,2,3,1,7,4,1,5,1,2,8,1
